<a href="https://colab.research.google.com/github/Winter-Shade/tutorial-Pytorch/blob/main/Text_2_SQL_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import torch
from datasets import load_dataset


ds = load_dataset("gretelai/synthetic_text_to_sql")

In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
        num_rows: 5851
    })
})

In [10]:
train_dataset = ds["train"]
test_dataset = ds["test"]

In [22]:
def format_input(entry):
  prompt = f"""Below is a question that describes a sql query, paired with a context that provides further information.\nWrite an answer that appropriately completes the sql query.\n\n### Question:\n{entry['sql_prompt']}"""

  context = f"""\n\n### Context:\n{entry["sql_context"] if entry["sql_context"] else ""}\n"""

  return prompt + context

In [23]:
model_input = format_input(train_dataset[20])
desired_response = f"\n\n### Response: {train_dataset[20]['sql']}"

print(model_input + desired_response)

Below is a question that describes a sql query, paired with a context that provides further information.
Write an answer that appropriately completes the sql query.

### Question:
What is the total number of autonomous driving research papers published by country in the past year?

### Context:
CREATE TABLE ResearchPapers (ID INT, Title VARCHAR(100), PublishedDate DATE, Author VARCHAR(50), Country VARCHAR(50)); INSERT INTO ResearchPapers (ID, Title, PublishedDate, Author, Country) VALUES (1, 'AD Research 1', '2022-01-15', 'A. Smith', 'USA'), (2, 'AD Research 2', '2022-03-20', 'B. Johnson', 'Canada'), (3, 'AD Research 3', '2021-12-12', 'C. Lee', 'South Korea'), (4, 'AD Research 4', '2022-05-08', 'D. Patel', 'India'), (5, 'AD Research 5', '2021-11-01', 'E. Chen', 'China');


### Response: SELECT Country, COUNT(*) as Total_Papers FROM ResearchPapers WHERE PublishedDate >= DATE_SUB(CURRENT_DATE, INTERVAL 1 YEAR) AND Author LIKE '%autonomous%' GROUP BY Country;


In [24]:
import torch
from torch.utils.data import Dataset

class Text2SqlDataset(Dataset):
  def __init__(self, data, tokenizer):
    self.data = data

    #Pre-tokenize inputs
    self.encoded_inputs = []
    for entry in data:
      prompt_plus_context = format_input(entry)
      response = f"\n\n### Response: {entry['sql']}"
      full = prompt_plus_context + response
      self.encoded_inputs.append(
          tokenizer.encode(full)
      )

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    return self.encoded_inputs[index]

In [29]:
%pip install --upgrade tiktoken -q
%pip install --upgrade openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.2/473.2 kB 23.3 MB/s eta 0:00:00


In [34]:
import tiktoken
tokenizer = tiktoken.get_encoding("p50k_base")
tokenizer = tiktoken.encoding_for_model("text-davinci-002")

print(tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"}))

[50256]


### Custom collate function --- finds the longest input in the batch and pads the other inputs with the end of text token;    
also convert list of inputs to tensors and transfer to target device

In [35]:
def custom_collate_draft_1(batch, pad_token_id=50256, device="cpu"):

  #find the longest sequence in the batch
  #and increase the max length by +1, which will add one extra
  #padding token below

  batch_max_length = max(len(item)+1 for item in batch)

  #pad and prepare inputs
  inputs_lst = []

  for item in batch:
    new_item = item.copy()
    new_item += [pad_token_id]

    padded = (
        new_item + [pad_token_id] * (batch_max_length - len(new_item))
    )

    inputs = torch.tensor(padded[:-1])
    inputs_lst.append(inputs)

  inputs_tensor = torch.stack(inputs_lst).to(device)
  return inputs_tensor

In [36]:
input1 = [1, 2, 3, 4, 5]
input2 = [6, 7]
input3 = [8, 9, 10]

batch = (
    input1,
    input2,
    input3
)

print(custom_collate_draft_1(batch))

tensor([[    1,     2,     3,     4,     5],
        [    6,     7, 50256, 50256, 50256],
        [    8,     9,    10, 50256, 50256]])


In [40]:
def custom_collate_draft_2(batch, pad_token_id=50256, device="cpu"):
  batch_max_length = max(len(item)+1 for item in batch)

  #pad and prepare inputs
  inputs_lst, targets_lst = [], []

  for item in batch:
    new_item = item.copy()

    new_item += [pad_token_id]

    padded = (
        new_item + [pad_token_id] * (batch_max_length - len(new_item))
    )

    inputs = torch.tensor(padded[:-1])
    targets = torch.tensor(padded[1:])
    inputs_lst.append(inputs)
    targets_lst.append(targets)

  inputs_tensor = torch.stack(inputs_lst).to(device)
  targets_tensor = torch.stack(outputs_lst).to(device)
  return inputs_tensor, targets_tensor

In [43]:
def custom_collate_fn(batch, pad_token_id=50256, ignore_index=-100, allowed_max_length=None,device="cpu"):
  batch_max_length = max(len(item)+1 for item in batch)

  #pad and prepare inputs
  inputs_lst, targets_lst = [], []

  for item in batch:
    new_item = item.copy()

    new_item += [pad_token_id]

    padded = (
        new_item + [pad_token_id] * (batch_max_length - len(new_item))
    )

    inputs = torch.tensor(padded[:-1])
    targets = torch.tensor(padded[1:])

    mask = targets == pad_token_id
    indices = torch.nonzero(mask).squeeze()

    if indices.numel() > 1:
      targets[indices[1:]] = ignore_index

    if allowed_max_length is not None:
      inputs = inputs[:allowed_max_length]
      targets = targets[:allowed_max_length]

    inputs_lst.append(inputs)
    targets_lst.append(targets)

  inputs_tensor = torch.stack(inputs_lst).to(device)
  targets_tensor = torch.stack(targets_lst).to(device)
  return inputs_tensor, targets_tensor

In [44]:
input1 = [1, 2, 3, 4, 5]
input2 = [6, 7]
input3 = [8, 9, 10]

batch = (
    input1,
    input2,
    input3
)

print(custom_collate_fn(batch))

(tensor([[    1,     2,     3,     4,     5],
        [    6,     7, 50256, 50256, 50256],
        [    8,     9,    10, 50256, 50256]]), tensor([[    2,     3,     4,     5, 50256],
        [    7, 50256,  -100,  -100,  -100],
        [    9,    10, 50256,  -100,  -100]]))
